# Regresní metriky

V úvodním příkladu s krajinou jsme si metriku vymýšleli sami. Existuje 
samozřejmě řada standardním metrik, které se k hodnocení modelů používají.
Uvedeme si přehled nejnámějších z nich:

-  **MSE (Mean squared error)** - průměr sumy čterců odchylek požadovaných výstupů od predikovaných výstupů 

    $$
    MSE(y, \hat{y}) = \frac{1}{N} \sum_{i=1}^N (y_i - y_N)^2 
    $$

- **MAE (Mean absolute error)** - průměr absolutních hodnot odchylek požadovaných výstupů od predikovaných výstupů 

    $$
    MAE(y, \hat{y}) = \frac{1}{N} \sum_{i=1}^N |y_i - y_N| 
    $$

- **$R^2$ score** 